In [3]:
%matplotlib inline 
#有了matplotlib inline 就可以省掉plt.show()了
%load_ext autoreload
%autoreload 2

from __future__ import division
from imports import *
#import pylab #使用pylab模式，可以直接使用numpy、pandas和绘图，而不需要再次引入,会配置为使用你所指定的matplotlib gui后端，还会提供一种更接近于matlab的界面
#%pylab
from IPython.display import display_html
display_html("""<button onclick="$('.input, .prompt, .output_stderr, .output_error').toggle();">Toggle code</button>""", raw=True)
# 允许多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from sklearn import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Toggle code

## 审查分类算法

### 算法审查
通过一系列实验判断出哪种算法对问题最有效，然后再进一步来选择算法  
应该先猜测一下，什么算法会具有最好的效果，这是训练我们对数据敏感性的好方法

### 线性算法
逻辑回归和线性判别分析都是假定输入的数据符合高斯分布

#### 逻辑回归
利用已知的自变量来预测一个离散型因变量的值（如二进制值0/1、是/否、真/假）。其实是一个分类算法而不是回归算法。预测的是一个概率值

In [11]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

filename = './csvs/user_label_7day_0301.csv'
datasource = pd.read_csv(filename)
#datasource.head(10)
data = datasource[datasource['user_label_7day.fplatformname']=='新浪平台']
#data.shape
array = data.values
X = array[:,5:9].astype(np.int64)
Y = array[:,9].astype(np.int64)
num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds,random_state=seed)
model = LogisticRegression()
result = cross_val_score(model,X,Y,cv=kfold)
print result

[ 0.96875    0.9625     0.95       0.95625    0.9375     0.96875    0.9625
  0.925      0.95       0.9245283]


#### LDA，是模式识别的经典算法 
将高维的模式样本投影到最佳鉴别矢量空间，以达到抽取分类信息和压缩特征空间维数的效果  
投影后保证模式样本在新的子空间有最大的类间距离和最小的类内距离

In [14]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis()
result = cross_val_score(model,X,Y,cv=kfold)
result.mean()

0.93682783018867932

### 非线性算法
#### K近邻算法
通过计算对象间距离来作为各个对象之间的非相似性指标  
同时，KNN通过依据k个对象中占优的类别进行决策

In [15]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
result = cross_val_score(model,X,Y,cv=kfold)
result.mean()

0.95245283018867932

#### 贝叶斯分类器 
原理是通过某对象的先验概率，利用贝叶斯公式，计算出后验概率，即该对象属于某一类的概率  
对输入数据同样做了符合高斯分布的假设

In [16]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
result = cross_val_score(model,X,Y,cv=kfold)
result.mean()

0.89430424528301899

#### 分类与回归树（CART）
对回归树用平方误差最小化（或基尼指数），进行特征选择，生成二叉树

In [17]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
result = cross_val_score(model,X,Y,cv=kfold)
result.mean()

0.90680817610062903

#### 支持向量机（SVM）

In [18]:
from sklearn.svm import SVC
model = SVC()
result = cross_val_score(model,X,Y,cv=kfold)
result.mean()

0.95245283018867932

## 审查回归算法
### 线性算法
#### 线性回归算法
运用两种或两种以上的变量间相互依赖的定量关系的一种统计分析方法  
$y=wx+e$,e表示误差服从均值为0的正态分布。只有一个自变量和因变量的，叫一元线性回归，两个以上的，且因变量和自变量之间是线性关系，则称为多元线性回归

In [19]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
scoring = 'neg_mean_squared_error'
result = cross_val_score(model,X,Y,cv=kfold,scoring=scoring)
result.mean()

-5.5558239047559947

#### 岭回归算法
实际是一种改良的最小二乘估计法，通过放弃最小二乘法的无偏性，以损失部分信息、降低精度为代价，获得回归系数

In [21]:
from sklearn.linear_model import Ridge
model = Ridge()
result = cross_val_score(model,X,Y,cv=kfold,scoring=scoring)
result.mean()

-5.5558185998882816

#### 套索回归算法

In [22]:
from sklearn.linear_model import Lasso
model = Lasso()
result = cross_val_score(model,X,Y,cv=kfold,scoring=scoring)
result.mean()

-5.5708424929521065

#### 弹性网络回归算法
是套索回归算法和岭回归算法的混合体

In [24]:
from sklearn.linear_model import ElasticNet
model = ElasticNet()
result = cross_val_score(model,X,Y,cv=kfold,scoring=scoring)
result.mean()

-5.5641633739214473

### 非线性算法
#### K近邻算法

In [26]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor()
result = cross_val_score(model,X,Y,cv=kfold,scoring=scoring)
result.mean()

-6.4392849056603767

#### 分类与回归树

In [27]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
result = cross_val_score(model,X,Y,cv=kfold,scoring=scoring)
result.mean()

-11.505887359167623

#### 支持向量机

In [28]:
from sklearn.svm import SVR
model = SVR()
result = cross_val_score(model,X,Y,cv=kfold,scoring=scoring)
result.mean()

-5.758763297592326

> 算法评估矩阵的计算经常使用cross_val_score函数，通过制定scoring参数来选择使用不同的评估矩阵